In [1]:
import http.client
import urllib.parse
import pandas as pd
import json

API_ENDPOINT = "api.open-meteo.com"
PATH = "/v1/forecast"
LAT = "57.682089684462454"
LONG = "11.983288826091862"

params = {
    "latitude": LAT,
    "longitude": LONG,
    "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m,precipitation,visibility,snowfall"
}
params_str = urllib.parse.urlencode(params)
url = f"{PATH}?{params_str}"

conn = http.client.HTTPSConnection(API_ENDPOINT)
conn.request("GET", url)
response = conn.getresponse()

if response.status == 200:
    response_data = response.read()
    weather_data = json.loads(response_data)
    hourly_data = weather_data["hourly"]
    data_dict = {
        'Temperature': [hourly_data['temperature_2m'][0]],
        'Humidity': [hourly_data['relativehumidity_2m'][0]],
        'Wind_Speed': [hourly_data['windspeed_10m'][0]],
        'Precipitation': [hourly_data['precipitation'][0]],
        'Visibility': [hourly_data['visibility'][0]],
        'Snowfall': [hourly_data['snowfall'][0]]
    }
    weather_df = pd.DataFrame(data_dict)
else:
    print("Failed to retrieve weather data.")

weather_df


,Temperature,Humidity,Wind_Speed,Precipitation,Visibility,Snowfall
0,4.5,80,11.9,0.0,24140.0,0.0


In [3]:
import os
from bike_data_scraper.s3_client.s3_handler import S3Handler

# Setup
WEATHER_KEY = os.environ["WEATHER_KEY"]
BIKES_KEY = os.environ["BIKES_KEY"]
SOURCE_BUCKET = os.environ["S3_SOURCE_BUCKET"]
s3_handler = S3Handler()

def lambda_handler(event, context):
    try:
        weather_data, bikes_data = fetch_data_from_s3()
        # Now weather_data and bikes_data are Pandas dataframes
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise e

def fetch_data_from_s3():
    weather_data = s3_handler.get_data_from_s3(SOURCE_BUCKET, WEATHER_KEY)
    bikes_data = s3_handler.get_data_from_s3(SOURCE_BUCKET, BIKES_KEY)

    if weather_data is None or bikes_data is None:
        raise ValueError("Missing data for weather or bikes")

    return weather_data, bikes_data



ModuleNotFoundError: No module named 'bike_data_scraper'